In [1]:
%pip install openpyxl
%pip install os
%pip install pytesseract
%pip install opencv-python
%pip install gspread
%pip install google-api-python-client


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#to import the data from google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Replace these with your credentials and spreadsheet information
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\mmcgh\OneDrive\Desktop\VS Codes\linking-sheets-and-colab-8c7e3059f5f9.json', scope)
spreadsheet_id = '1IyR7x4PtVEl__WEe2RSt45w6-aMQvqQnQy_pTjgMKro'
worksheet_name = 'test'  # Change this to the name of your worksheet

# Authenticate with Google Sheets API
gc = gspread.authorize(credentials)

# Open the worksheet
worksheet = gc.open_by_key(spreadsheet_id).worksheet(worksheet_name)

# Get all values from the worksheet
rows = worksheet.get_all_values()

# Convert to DataFrame
df_initial_data = pd.DataFrame(rows[1:], columns=rows[0])

def extract_file_id(link):
    return link.split('=')[-1]

# Apply the function to the 'link' column
df_initial_data['file_id'] = df_initial_data['Link'].apply(extract_file_id)

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload  # Import MediaIoBaseDownload
import io

credentials_file = r'C:\Users\mmcgh\OneDrive\Desktop\VS Codes\linking-sheets-and-colab-8c7e3059f5f9.json'

for i in range(len(df_initial_data['file_id'])):
    file_id = df_initial_data['file_id'][i]
    invoice_number = df_initial_data['Invoice No.'][i]  # Access each invoice number individually
    #trimmed_invoice = invoice_number[-4:]  # Slice the invoice number to get the last four digits
    custom_filename = r'C:\Users\mmcgh\OneDrive\Desktop\VS Codes\extracting text from image\Dataset\{}.pdf'.format(invoice_number)
    credentials = service_account.Credentials.from_service_account_file(credentials_file)
    drive_service = build('drive', 'v3', credentials=credentials)

    # Download the file
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

    # Save the downloaded file with custom filename
    with open(custom_filename, 'wb') as f:
        fh.seek(0)
        f.write(fh.read())

    print(invoice_number+" downloaded successfully.")

# Replace these with your credentials and spreadsheet information
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\mmcgh\OneDrive\Desktop\VS Codes\linking-sheets-and-colab-8c7e3059f5f9.json', scope)
spreadsheet_id = '1IyR7x4PtVEl__WEe2RSt45w6-aMQvqQnQy_pTjgMKro'
worksheet_name = 'test'  # Change this to the name of your worksheet

# Authenticate with Google Sheets API
gc = gspread.authorize(credentials)

# Open the worksheet
worksheet = gc.open_by_key(spreadsheet_id).worksheet(worksheet_name)

# Get all values from the worksheet
rows = worksheet.get_all_values()

# Convert to DataFrame
df_initial_data = pd.DataFrame(rows[1:], columns=rows[0])

# Display DataFrame
df_initial_data

import pandas as pd
import os
import re
import PyPDF2

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    try:
        text = ""
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

def find_file(file_name, search_path="."):
    """
    Search for a file by name in the specified directory and its subdirectories.
    """
    try:
        for root, dirs, files in os.walk(search_path):
            if file_name in files:
                return os.path.join(root, file_name)
        return None
    except Exception as e:
        print(f"Error searching for file: {e}")
        return None

def process_pdf_for_invoice(invoice_number, search_path):
    """Process PDF files corresponding to an invoice number."""
    file_name_to_search = f"{invoice_number}.pdf"
    found_file_path = find_file(file_name_to_search, search_path)
    if found_file_path:
        try:
            extracted_text = extract_text_from_pdf(found_file_path)
            match_party_name = re.findall(r'Party Name\s*:\s*(.*)', extracted_text)
            match_phone_number = re.findall(r'PHONE\.?\s*:\s*([\d\s,-/]+)', extracted_text)
            match_invoice_number = re.findall(r'Invoice No (\w+)', extracted_text)
            #match_loc = re.findall(r'PHONE')
            #return match_party_name, [num.strip('\n').split(',') for num in match_phone_number], match_invoice_number
            # Extract last 10 digits of each phone number
            last_10_digits_phone = [phone[-11:] for phone in match_phone_number]
            
            return match_party_name, last_10_digits_phone, match_invoice_number
        except Exception as e:
            print(f"Error processing PDF for invoice number {invoice_number}: {e}")
            return None, None, None
    else:
        print(f"PDF file not found for invoice number {invoice_number}")
        return [], [], []  # Return empty lists if file is not found


# Define the column name or index you want to extract
column_name = "Invoice No."  # Replace "Column_Name" with the name of your column

# Specify the directory to start searching PDF files
search_path = r"C:\Users\mmcgh\OneDrive\Desktop\VS Codes\extracting text from image\Dataset"

# Initialize lists to store extracted data
party_name = []
phone_num = []
invoice_num = []

try:
    for invoice_number in df_initial_data[column_name]:
        party, phone, invoice = process_pdf_for_invoice(invoice_number, search_path)
        party_name.append(party)
        phone_num.append(phone)
        invoice_num.append(invoice)
except Exception as e:
    print(f"Error processing invoices: {e}")

# Flatten the lists (handle empty lists)
party_name_flat = [item for sublist in party_name for item in sublist]
phone_num_flat = [item for sublist in phone_num for item in sublist]
invoice_num_flat = [item for sublist in invoice_num for item in sublist]

# Create a dictionary from the flattened lists
data = {
    'Party Name': party_name_flat,
    'Phone Number': phone_num_flat,
    'Invoice Number': invoice_num_flat
}

# Create DataFrame
df_final_data = pd.DataFrame(data)

# Print the DataFrame
df_final_data

#appending to the list
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Replace these with your credentials and spreadsheet information
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\mmcgh\OneDrive\Desktop\VS Codes\linking-sheets-and-colab-8c7e3059f5f9.json', scope)
spreadsheet_id = '1IyR7x4PtVEl__WEe2RSt45w6-aMQvqQnQy_pTjgMKro'
worksheet_name = 'demo'  # Change this to the name of your worksheet

# Authenticate with Google Sheets API
gc = gspread.authorize(credentials)

# Open the worksheet
worksheet = gc.open_by_key(spreadsheet_id).worksheet(worksheet_name)

# Load your DataFrame in VSCode
# df_final_data = ...  # Your DataFrame loaded in VSCode

# Flatten the lists in the "Phone Number" column
df_final_data['Phone Number'] = df_final_data['Phone Number'].apply(lambda x: ''.join(map(str, x)))

# Append DataFrame to Google Sheets
existing_rows = worksheet.get_all_values()
existing_df = pd.DataFrame(existing_rows[1:], columns=existing_rows[0])
updated_df = pd.concat([existing_df, df_final_data], ignore_index=True)

# Clear existing content and append updated DataFrame
worksheet.clear()
worksheet.update([updated_df.columns.values.tolist()] + updated_df.values.tolist())
